In [1]:
import os
from openai import OpenAI
from pydantic import BaseModel, Field
import json

client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.getenv('NB_STUDIO_API_KEY')
)

model="meta-llama/Meta-Llama-3.1-8B-Instruct-fast"



In [2]:
class PersonOutput(BaseModel):
    first_name: str
    surname: str
    role: str = Field(description = 'Role of the person in the company.')

schema = PersonOutput.schema()

text = """
The task of json generation was assigned Mikhail Kashirskii, a software developer at Nebius. \
His role in this task is to implement the support of json outputs of the LLM.
""".strip()

system_prompt = f"""
Act as an experienced data extractor. \
Your goal is to extract the required information (is given below) from the text. \
Output the extracted information in the json format, following the required schema.
Required information schema:
```
{schema}
```
""".strip()

completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ],
    extra_body={
        "guided_json": schema,
    },

    temperature=0.6,
    max_tokens=51,
    top_p=0.9
)

print(completion.to_json())
person = PersonOutput(**json.loads(completion.choices[0].message.content))
assert person.first_name == 'Mikhail'

{
  "id": "chat-4d4d16562cf44fe5b44b07c2235e7a1c",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\"first_name\": \"Mikhail\", \"surname\": \"Kashirskii\", \"role\": \"Software Developer\"}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": []
      },
      "stop_reason": null
    }
  ],
  "created": 1725969933,
  "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-fast",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 187,
    "total_tokens": 213
  },
  "prompt_logprobs": null
}


In [3]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ],
    response_format={ "type": "json_object" },

    temperature=0.6,
    max_tokens=151,
    top_p=0.9
)

print(completion.to_json())
person = PersonOutput(**json.loads(completion.choices[0].message.content))
assert person.first_name == 'Mikhail'


{
  "id": "chat-0ca99f6a8fc9447690ff0a5e4b866954",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{ \n   \t\"first_name\" \t: \t\"Mikhail\" \t, \n\t\"surname\" \t: \t\"Kashirskii\" \t, \n\t\"role\" \t: \t\"Software developer at Nebius\" \n}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": []
      },
      "stop_reason": null
    }
  ],
  "created": 1725969934,
  "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-fast",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 59,
    "prompt_tokens": 187,
    "total_tokens": 246
  },
  "prompt_logprobs": null
}


In [4]:
import json

system_prompt = f"""
Based on the person's description, classify whether the person has relation to AI or not. \
Output only "True" if the person has relation to IT or "False", otherwise.
""".strip()

text = """
Mikhail Kashirskii, a software developer at Nebius, was assigned the task of json generation. \
His role in this task is to implement the support of json outputs of the LLM.
""".strip()

completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ],
    temperature=0.,
    extra_body={
        "guided_choice": ["True", "False"]
    }
)

assert completion.choices[0].message.content == 'True'